<div class="pagebreak"></div>

# Classes: Inheritance
With inheritance, we can create a new class by extending an existing class. The new class inherits the existing class's attributes and methods (the members). We can then add attributes and/or methods. We can also modify existing attributes and methods to change functionality.

As we look around the world, we can find many instances of hierarchy in which specialized classes exist.  For example, the animal taxonomy serves to classify animals into different classes ([taxonomic ranks](https://en.wikipedia.org/wiki/Taxonomic_rank) based upon the different attributes that animals possess. Planes, cars, and trains are specialized cases of vehicles.  Circles, triangles, and rectangles are specialized cases of shapes. You can also view these examples using the phrase "<i>is-a</i>".  A train is a vehicle.  A circle is a shape. A dog is a mammal.

In object-oriented programming, inheritance creates this "_is-a_" relationship among classes. We build a new class from an existing class. The existing (original) class is called a parent, superclass, or base class. The superclass is the more general class. The new class is called a child, subclass, or derived class. This new class is the more specialized class. The subclass becomes specialized by adding attributes and/or methods. The subclass can also become more specialized by modifying the existing state or behavior. 


In the previous notebook, we defined a BankAccount.  However, many different types of bank accounts exist: checking, savings, money market, brokerage, etc.  We also have transactions that can belong to those accounts.  Again, though, we have different transactions types: deposit, withdrawal, transfer, purchase, etc.  Then under deposit transactions, we have additional specialties: check, cash, ACH, etc.  Similar specialties will also exist for the other transaction types.

As another example, consider different types of employees within a corporation. Corporations can have hourly employees, paid a fixed amount per hour and a specific multiplier for specialty shifts (holiday, weekend, nights). Corporations also can have salaried employees who earn a particular amount per pay period and commission employees who make a base salary per pay period plus a percentage of the gross sales they generate.

The below diagram is a Unified Modeling Language(UML) class diagram. These diagrams document classes, their relationships to each other, and each class's members (attribute and methods). Each box represents a class. The top line contains that class name. The second area in the box contains the attributes, and the third area contains the methods.

![](images/EmployeeClassHierarchy.png)

Objects of the `Employee` class have three attributes - an ID, a name, and a job title - and no methods are defined. The `HourlyEmployee` class extends the `Employee` class by adding attributes for the employee's hourly rate and the hours that the employee worked during the current pay period. The `HourlyEmployee` also adds behavior - a method to compute their pay. The `SalariedEmployee` class extends the `Employee` class by adding an attribute for the employee's periodic salary amount. For example, if an employee earns $\$$120,000 annually and is paid twice a month, the periodic pay would be $\$$5,000. The `SalariedEmployee` class also defines a method to compute pay, but this differs from the HourlyEmployee's behavior. `CommissionedEmployee` inherits the behavior and attributes of `SalariedEmployee` but adds an attribute to track their sales over the past pay period. Their pay calculation will "override" the `SalariedEmployee` class's pay calculation as they receive a salary plus a percentage of their gross sales.

The following code cell defines the `Employee` class which will be the superclass for the three other employee classes.  This code should be familiar based on the previous notebook. This code does create a `calculate_pay()` method, but any call to that will raise an exception. By defining a method here, we establish that any subclass must implement a `calculate_pay()` method.

In [ ]:
class Employee:
    """Employee"""
    __id = 100  # must change name otherwise a recursive overflow error occurs
    
    def __init__(self, name, job_title):
        self.__id = Employee.__id
        Employee.__id += 1
        self.__name = name
        self.__job_title = job_title
        
    def __str__(self):
        return "ID #{:d}: {:s}({:s}, {:s})".format(self.id,self.name, self.job_title, self.employee_type)
    
    def __repr__(self):
        return str({ "id": self.id, "name": self.name, "job_title":self.job_title })
    
    @property
    def id(self):
        return self.__id

    @property
    def name(self):
        return self.__name
    
    @property
    def job_title(self):
        return self.__job_title
    
    
    @job_title.setter
    def job_title(self, new_title):
        self.__job_title = new_title
    
    @property
    def employee_type(self):
        return "unknown"
        
    
    def calculate_pay(self):
        raise  NotImplementedError("Employee subclasses must implement calculate_pay()")

Some sample code to perform ad-hoc testing that the class works.

In [ ]:
a = Employee("Steve","Programmer")
b = Employee("Christine","Project Manager")
print(a)
repr(b)

Now, define a more robust set of unit tests for `Employee`.

In [ ]:
import unittest

class TestEmployee(unittest.TestCase):
    def setUp(self):
        Employee._Employee__id = 100
        
    def test_create(self):
        import ast
        
        a = Employee("Steve","Programmer")
        b = Employee("Christine","Project Manager")
        self.assertNotEqual(a.id,b.id,"IDs are not unique")
        self.assertEqual(a.name,"Steve")
        self.assertEqual(b.name,"Christine")
        self.assertEqual(a.job_title,"Programmer")
        self.assertEqual(b.job_title,"Project Manager")
        self.assertEqual(str(a),"ID #100: Steve(Programmer, unknown)")
        self.assertEqual(ast.literal_eval(repr(b)),{'id': 101, 'name': 'Christine', 'job_title': 'Project Manager'})
        
    def test_calculate_pay_not_implemented(self):
        a = Employee("Steve","Programmer")
        with self.assertRaises(Exception) as context:
            a.calculate_pay()
        
        self.assertTrue(type(context.exception) == NotImplementedError)
        self.assertTrue("must implement" in context.exception.args[0])
        
    def test_change_job_title(self):
        a = Employee("Steve","Programmer")
        a.job_title = 'Senior Programmer'
        self.assertEqual(a.job_title,'Senior Programmer')
        self.assertEqual(str(a),"ID #100: Steve(Senior Programmer, unknown)")
        
unittest.main(argv=['unittest','TestEmployee'], verbosity=2, exit=False)

At this point, we have implemented our base `Employee` class and have a robust set of test cases for it.

Next, let's look at defining the `HourlyEmployee` class. The following code block adds a few new details:
1. We define `HourlyEmployee` as a subclass of `Employee`. Subclasses are defined similarly to other classes, except we add the parent class name inside of parenthesis at the end.  Syntax -
   <pre>
   class <i>ClassName</i>(<i>ParentClassName</i>):
   </pre>
2. In the initializer, we make a call to the parent class with `super()`.  As we have defined `__init__` in the child class, the interpreter does not automatically call the corresponding method in the parent class is not automatically called. Therefore, we explicitly call the initializer with reference to the superclass (`super()`). This call ensures our code performs the steps to initialize the base `Employee` type properly.  The `__init__` method then continues with setting the attributes specific to instances of `HourlyEmployee`.
3. Note that in the initializer, we explicitly set `__hours_worked` to `None`. This statement defines that attribute. In `calculate_pay()`, we add a sanity check to our code to ensure `hours_worked` has a valid value with the `assert` statement.  Programmers can place a conditional expression within an `assert` statement. If the expression evaluates to `True`, processing continues normally. If the expression evaluates to `False`, the interpreter raises an exception.
4. The `HourlyEmployee` class adds additional methods to support the `hourly_rate` and `hours_worked` attributes.
5. In addition to the `__init__` method, the `HourlyEmployee` class also overrides the methods for `employee_type` and `calculate_pay`.  By overriding methods, objects of type `HourlyEmployee` will use the behavior for the methods defined within  `HourlyEmployee` itself. Any behavior defined in the parent class will not be performed unless explicitly called through the `super()` reference.
6. Notice that the `HourlyEmployee` class did not change the `__str__` or `__repr__` methods.  In the test code in the following block, notice that when the `__str__` method gets the employee type, it calls the method based on the actual class. i.e., an instance of `Employee` returns "unknown" while an instance of `HourlyEmployee` returns "hourly".

In [ ]:
from decimal import Decimal

class HourlyEmployee(Employee):
    """Hourly Employee"""
    
    
    def __init__(self, name, job_title,hourly_rate):
        super().__init__(name,job_title)
        self.__hourly_rate = Decimal(hourly_rate)
        self.__hours_worked = None        
        
    @property
    def employee_type(self):
        return "hourly"
        
    @property
    def hourly_rate(self):
        return self.__hourly_rate
    
    
    @hourly_rate.setter
    def hourly_rate(self, new_rate):
        self.__hourly_rate= new_rate        
        

    @property
    def hours_worked(self):
        return self.__hours_worked
    
    
    @hours_worked.setter
    def hours_worked(self, new_hours):
        self.__hours_worked = Decimal(new_hours)               
        
    def calculate_pay(self):
        assert type(self.hours_worked) is Decimal, "Hours worked not established"
        hours = self.hours_worked
        overtime_hours = Decimal(0)
        if hours > 40:
            overtime_hours = hours - Decimal(40.0)
            hours = Decimal(40.0)
        return hours * self.hourly_rate + overtime_hours * self.hourly_rate* Decimal(1.5)

Run some code to see how the `HourlyEmployee` class works.

In [ ]:
c = HourlyEmployee("Max","System Administrator","54.76")
print(c)
print(c.name)
print(c.hours_worked)
print(c.calculate_pay())   # will cause an assertion error as hours_worked not set.

Now define some additional test cases. First, we check that the parent functionality still works.
We also check that the `calculate_pay()` method checks that hours_worked has a valid numerical value. We also check several equivalence  classes for `hours_worked` in `compute_pay()` to cover amounts < 40 hours, amounts equal to 40 hours, and amounts greater than 60 hours. Finally, we check that the employee type value is correct for the different types.

In [ ]:
import unittest

class TestHourlyEmployee(unittest.TestCase):
    def setUp(self):
        Employee._Employee__id = 100
        
    def test_create(self):
        a = HourlyEmployee("Max","System Administrator",65.0)
        self.assertEqual(a.name,"Max")
        self.assertEqual(a.job_title,"System Administrator")
        self.assertEqual(str(a),"ID #100: Max(System Administrator, hourly)")

    def test_compute_pay_no_hours(self):
        a = HourlyEmployee("Max","System Administrator",65.0)
        with self.assertRaises(Exception) as context:
            a.calculate_pay()
        
        self.assertTrue(type(context.exception) in [TypeError,AssertionError])

    def test_compute_pay(self):
        a = HourlyEmployee("Max","System Administrator",65.0)
        a.hours_worked = 20
        self.assertEqual(a.calculate_pay(), Decimal(1300.0), "Pay not correct")
        a.hours_worked = 40
        self.assertEqual(a.calculate_pay(), Decimal(2600.0), "Pay not correct")
        a.hours_worked = 60
        self.assertEqual(a.calculate_pay(), Decimal(4550.0), "Pay not correct")

        
    def test_compute_pay_with_overtime(self):
        a = HourlyEmployee("Max","System Administrator",65.0)
        
    def test_employee_types(self):
        a = HourlyEmployee("Max","System Administrator",65.0)
        b = Employee("Cindy", "Sales Manager")
        self.assertEqual(a.employee_type,"hourly")
        self.assertNotEqual(a.employee_type,b.employee_type)
        
unittest.main(argv=['unittest','TestHourlyEmployee'], verbosity=2, exit=False)

## Multiple Inheritance
Multiple inheritance is the ability of a class to inherit from two or more superclasses. 

The primary drawback to multiple inheritance is the diamond problem. In the below diagram, suppose classes A, B, and C have all defined a particular method while D has not. Then, when that method is called on an object of class D, which version of the method is used? A's? B's? C's?

![](images/DiamondProblem.png)

Python solves this problem by defining a specific method resolution order. When looking for a method or attribute, Python performs the following search: the object itself, the object's class, the first parent class, the second parent class, the n<sup>th</sup> parent class, and then those parent's in order.

For example, consider the following set of classes:

In [ ]:
class Employee:
    def beverage(self): return "water"
    def skill(self):    return "Works hard"

class Programmer(Employee):
    def skill(self):    return "Writes Code"

class Statistician(Employee):
    def skill(self):    return "statistical analysis"
    
class StoryTeller(Employee):
    def beverage(self): return "beer"
    def skill(self):    return "tells stories"
    
class ComputationalDataScientist(Programmer,Statistician):
    def beverage(self): return "Mountain Dew"

class StatisticalAnalyst(Statistician, Programmer):
    def beverage(self): return "tea"

class Presenter(StoryTeller, Programmer, Statistician):
    pass


In [ ]:
print("Skills - ")
print("ComputationalDataScientist:", ComputationalDataScientist().skill())
print("StatisticalAnalyst:", StatisticalAnalyst().skill())
print("Presenter:", Presenter().skill())
print("\Beverages - ")
print("ComputationalDataScientist:", ComputationalDataScientist().beverage())
print("StatisticalAnalyst:", StatisticalAnalyst().beverage())
print("Presenter:", Presenter().beverage())

Each Python class contains a method `mro()` that returns the list of classes to search to find a particular attribute or method for an object of that class.

In [ ]:
ComputationalDataScientist.mro()

As we look for a skill for a `ComputationalDataScientist`, the interpreter checks these class definitions
1. The object itself  
2. The object's class   (class and static methods)
3. The class's first parent class - `Programmer`
4. The class's second parent class - `Statistician`
5. It then continuex checking the parent's superclasses in a similar order.

Finding the `skill()` implementation for `StatisticalAnalyst` follows the same logic, but its first parent class is `Statistician` and then `Programmer`.

`Presenter` shows that we could inherit from 3 parent classes.

In [ ]:
Presenter.mro()

## Polymorphism and Duck Typing
Polymorphism is the ability to call a specific method (send a message) to an object without knowing the receiving object's actual type. If the receiving object implements that method then it can respond appropriately. If the receiving object does not implement that method, then a runtime exception is generated.

The `Employee` class demonstrates polymorphism within the `__str__` method. That method calls the `employee_type()` method without knowing the exact underlying type of self. The multiple inheritance example demonstrates polymorphism as well.

With Polymorphism, Python programmers can apply the same operation(method call) to different types - just as long as the method's name and the number of arguments exist with the receiving type's definition. This behavior is sometimes called [duck typing](https://en.wikipedia.org/wiki/Duck_typing).

Polymorphism allows methods of the same name to have predictable behavior but allows the underlying class to define the specific behavior independently.

## Mixins 
A popular use case for multiple inheritance is to inherit from a particular class (a "mixin") that defines well-established methods and attributes (features).  Usually, only one feature exists in a "mixin" class.  This class does not share methods with any other parent class - this avoids the diamond problem.  The inheritance from "mixins" is not an "is a" relationship, but rather "has behavior".

The methods in "mixin" classes are typically "side" tasks - sometimes generic in nature, such as logging or type conversions. However, the methods can also be specific to the problem domain, adding shared functionality to different classes. For example, a charting library could have mixins to deal with colors and legends.

The below example creates `DumpAttributeMixin` to print an object's attributes.

In [ ]:
class DumpAttributeMixin:
    def dump(self):
        import pprint
        pprint.pprint(vars(self))
        
class HourlyDumpEmployee(HourlyEmployee, DumpAttributeMixin):
    pass

c = HourlyDumpEmployee("Max","System Administrator","54.76")
c.dump()

## Note
In Python, all objects implicitly inherit from the class `object` if a parent class is not explicitly  defined. This implicit inheritance is `object` appears as the last item in the `mro()` calls above. This inheritance hierarchy allows for shared behavior defined among all created objects in Python - such as the ability to get the string representation.

In [ ]:
help(object)

## Exercises
1. Complete the Salaried and Commissioned Employee classes. Commissioned employees earn 5% of their sales. Salary should be set when constructing the classes.